<a href="https://colab.research.google.com/github/readwalkerchang/Non-Personalized-Suggestion-recommender/blob/main/Non_Personalized_Suggestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate Achievement dataset for testing

In [54]:
import pandas as pd  
import numpy as numpy
  
# assign data of lists.  
AchievementData = {'Achievement': ['Maker of the Month', 'Crashed It!', 'Christmas Savior', 'Learning the Basics']}  
userdata =  {'UserAchievement': ['Maker of the Month', 'Crashed It!']}  


# Create DataFrame  
dfAchievementData = pd.DataFrame(AchievementData)  
dfuserdata = pd.DataFrame(userdata)  
  
# Print the output.  
print(dfAchievementData)  
print(dfuserdata)  

           Achievement
0   Maker of the Month
1          Crashed It!
2     Christmas Savior
3  Learning the Basics
      UserAchievement
0  Maker of the Month
1         Crashed It!


In [55]:
from itertools import permutations
def create_pairs(item_df):
    pairs = permutations(item_df.values, 2) # Randomly draw 2 items from all items without putting them back 
    pairs = list(pairs) # Because the permutation here is actually an iterator, it needs to be instantiated with a list 
    pairs = pd.DataFrame(pairs, columns=["items_A", "items_B"])
    return pairs

UserAchievementPairs = create_pairs(dfuserdata)
AchievementPairs = create_pairs(dfAchievementData)
print(AchievementPairs,'\n',UserAchievementPairs)

                  items_A                items_B
0    [Maker of the Month]          [Crashed It!]
1    [Maker of the Month]     [Christmas Savior]
2    [Maker of the Month]  [Learning the Basics]
3           [Crashed It!]   [Maker of the Month]
4           [Crashed It!]     [Christmas Savior]
5           [Crashed It!]  [Learning the Basics]
6      [Christmas Savior]   [Maker of the Month]
7      [Christmas Savior]          [Crashed It!]
8      [Christmas Savior]  [Learning the Basics]
9   [Learning the Basics]   [Maker of the Month]
10  [Learning the Basics]          [Crashed It!]
11  [Learning the Basics]     [Christmas Savior] 
                 items_A               items_B
0  [Maker of the Month]         [Crashed It!]
1         [Crashed It!]  [Maker of the Month]


# Add counts to achievement pairs

In [56]:
#create ramdom number and add a row 'count' for eash pairs
import random
randomlist = []
for i in range(0,12):
  n = random.randint(1,30)
  randomlist.append(n)
AchievementPairs['count'] = randomlist
AchievementPairs

,items_A,items_B,count
0,[Maker of the Month],[Crashed It!],18
1,[Maker of the Month],[Christmas Savior],24
2,[Maker of the Month],[Learning the Basics],24
3,[Crashed It!],[Maker of the Month],7
4,[Crashed It!],[Christmas Savior],29
5,[Crashed It!],[Learning the Basics],1
6,[Christmas Savior],[Maker of the Month],3
7,[Christmas Savior],[Crashed It!],11
8,[Christmas Savior],[Learning the Basics],27
9,[Learning the Basics],[Maker of the Month],28


#Clean the pair list

 Get the number of rows, columns and elements


```
Get the number of columns: len(df.columns)
Get the number of rows and columns: df.shape
Get the number of elements: df.size
```


Loop through the AchievementPairs and find if the items_A match users items_A by using 

```
update_df.loc[1,"items_A"]
```


If it matches, then see which pair also match item_B
If it matches both, delete this row by using:


```
update_df = AchievementPairs.drop(0)
```





In [63]:
def remove_duplicated_pairs(user_df, service_df):
  RemoveList = []
  for i in range(len(user_df)) :
    for j in range(len(service_df)) :
      if user_df.loc[i, "items_A"] == service_df.loc[j, "items_A"]:
        if user_df.loc[i, "items_B"] == service_df.loc[j, "items_B"]:
          RemoveList.append(j)
  return RemoveList


RemoveList = remove_duplicated_pairs(UserAchievementPairs,AchievementPairs)
print("Remove:",RemoveList)
update_AchievementPairs = AchievementPairs.drop(RemoveList).reset_index(drop=True)
print(update_AchievementPairs)

Remove: [0, 3]
                 items_A                items_B  count
0   [Maker of the Month]     [Christmas Savior]     24
1   [Maker of the Month]  [Learning the Basics]     24
2          [Crashed It!]     [Christmas Savior]     29
3          [Crashed It!]  [Learning the Basics]      1
4     [Christmas Savior]   [Maker of the Month]      3
5     [Christmas Savior]          [Crashed It!]     11
6     [Christmas Savior]  [Learning the Basics]     27
7  [Learning the Basics]   [Maker of the Month]     28
8  [Learning the Basics]          [Crashed It!]     22
9  [Learning the Basics]     [Christmas Savior]      5


# Drop other pair which are not contain use's input

In [69]:
CleanedAchievementData = pd.DataFrame( columns=["items_A", "items_B", "count"] ) 

# for i in range(len(UserAchievementPairs)) :
#   for j in range(len(update_AchievementPairs)) :
#     if UserAchievementPairs.loc[i, "items_A"] == update_AchievementPairs.loc[j, "items_A"]:
#       iteamA = update_AchievementPairs.loc[j, "items_A"]
#       iteamB = update_AchievementPairs.loc[j, "items_B"]
#       count = update_AchievementPairs.loc[j, "count"]
#       newRow = {'items_A': iteamA, 'items_B': iteamB, 'count': count}
#       CleanedAchievementData = CleanedAchievementData.append(newRow ,ignore_index = True)


def create_candidates(user_df, service_df):
  CleanedAchievementData = pd.DataFrame( columns=["items_A", "items_B", "count"] ) 
  for i in range(len(user_df)) :
    for j in range(len(service_df)) :
      if user_df.loc[i, "items_A"] == update_AchievementPairs.loc[j, "items_A"]:
        iteamA = service_df.loc[j, "items_A"]
        iteamB = service_df.loc[j, "items_B"]
        count = service_df.loc[j, "count"]
        newRow = {'items_A': iteamA, 'items_B': iteamB, 'count': count}
        CleanedAchievementData = CleanedAchievementData.append(newRow ,ignore_index = True)
  return CleanedAchievementData

candidate_pairs = create_candidates(UserAchievementPairs, update_AchievementPairs)
print(candidate_pairs)

                items_A                items_B count
0  [Maker of the Month]     [Christmas Savior]    24
1  [Maker of the Month]  [Learning the Basics]    24
2         [Crashed It!]     [Christmas Savior]    29
3         [Crashed It!]  [Learning the Basics]     1


#Calculating recommendation

In [73]:
# Doing sorting

candidate_pairs_sorted = candidate_pairs.sort_values('count', ascending=False)
print(candidate_pairs_sorted)

                items_A                items_B count
2         [Crashed It!]     [Christmas Savior]    29
0  [Maker of the Month]     [Christmas Savior]    24
1  [Maker of the Month]  [Learning the Basics]    24
3         [Crashed It!]  [Learning the Basics]     1
